In [1]:
import pygmo as pg
import numpy as np

from f3dasm.simulation.benchmark_functions import Levy
from f3dasm.src.optimization import PygmoProblem
from f3dasm.src.designofexperiments import DoE
from f3dasm.src.space import ContinuousSpace

In [2]:
# Input parameters
x1 = ContinuousSpace(name="x1", lower_bound=-10., upper_bound=10.)
x2 = ContinuousSpace(name="x2", lower_bound=-10., upper_bound=10.)
x3 = ContinuousSpace(name="x3", lower_bound=-10., upper_bound=10.)

# Output parameters
y = ContinuousSpace(name="y")

input_space = [x1, x2, x3]
output_space = [y]

# Construct the design space
design = DoE(input_space=input_space, output_space=output_space)

In [3]:
algorithm = pg.algorithm(pg.cmaes(gen=1, memory=True, seed=42, force_bounds=True))
levy = Levy()

prob = pg.problem(PygmoProblem(design=design, func=levy))

In [4]:
prob
pop = pg.population(prob, size = 5)
pop

ValueError: 
function: uniform_real_from_range_checks
where: /home/conda/feedstock_root/build_artifacts/pagmo_1656419683381/work/include/pagmo/utils/generic.hpp, 69
what: Cannot generate a random real if the bounds are not finite


In [ ]:
for i in range(10000):
    pop = algorithm.evolve(pop)

pop

In [ ]:
uda = algorithm.extract(pg.cmaes)

In [ ]:
prob.get_name()